# 104: Model Interpretability & Explainability

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** the difference between interpretability (global understanding) and explainability (local predictions)
- **Implement** SHAP (SHapley Additive exPlanations) values for model-agnostic explanations
- **Build** LIME (Local Interpretable Model-agnostic Explanations) for individual prediction analysis
- **Apply** interpretability techniques to semiconductor yield prediction models
- **Evaluate** feature importance using multiple methods (permutation, tree-based, SHAP)

## 📚 What is Model Interpretability?

Model interpretability is the ability to understand and explain how a machine learning model makes decisions. In regulated industries like semiconductor manufacturing, healthcare, and finance, interpretability isn't optional—it's mandatory. A black-box model that predicts 92% yield but can't explain why is often less valuable than a 88% accurate model that clearly identifies root causes.

**Interpretability** answers "How does the model work globally?" while **Explainability** answers "Why did the model make this specific prediction?" For example, knowing that voltage is the most important feature overall (interpretability) helps design better tests, while knowing that Device #47251 failed because its Vdd was 3.2σ above spec (explainability) enables targeted debug.

Modern interpretability techniques like SHAP and LIME work with any model—from simple linear regression to complex neural networks. They transform opaque algorithms into transparent decision systems that engineers can validate, debug, and trust in production environments where mistakes cost millions.

**Why Model Interpretability?**
- ✅ **Regulatory Compliance**: FDA, automotive (ISO 26262), and finance require explainable AI for safety-critical decisions
- ✅ **Debug Model Failures**: Identify when models rely on spurious correlations or data leakage instead of real physics
- ✅ **Build Trust**: Engineers accept ML predictions when they understand the reasoning ("Yes, Vdd is too high")
- ✅ **Feature Engineering**: Discover which engineered features actually help vs add noise
- ✅ **Root Cause Analysis**: Pinpoint why specific devices fail ("Lot C fails due to high temperature + low Vth combination")

## 🏭 Post-Silicon Validation Use Cases

**Use Case 1: Yield Loss Root Cause Identification**
- **Input**: Failed devices from wafer test with 150+ parametric measurements
- **Model Output**: Binary classification (pass/fail) with 94% accuracy
- **Explainability**: SHAP values identify top 3 contributing parameters per failed die
- **Value**: Reduces root cause analysis time from 2 weeks to 2 days, $500K-$2M savings per yield excursion

**Use Case 2: Outlier Device Investigation**
- **Input**: Device #X has abnormal power consumption (2.5σ above mean)
- **Model Output**: Power prediction model with 95% R²
- **Explainability**: LIME shows Idd (leakage current) contributed +18mW, Freq contributed +7mW
- **Value**: Identifies design vs manufacturing issues, prevents false RMAs, saves $50K per misdiagnosed unit

**Use Case 3: Test Program Optimization Justification**
- **Input**: Random Forest model predicting bin outcome from 500 parametric tests
- **Model Output**: 97% bin prediction accuracy
- **Explainability**: Permutation importance shows 200 tests have zero impact on predictions
- **Value**: Safely remove redundant tests, reduce test time 35%, $3M-$7M annual ATE savings with proof

**Use Case 4: Adaptive Test Insertion Confidence**
- **Input**: Real-time wafer test results, model predicts next test's outcome
- **Model Output**: Probabilistic prediction (85% likely to pass AC parametric tests)
- **Explainability**: SHAP force plot shows DC tests already indicate marginal performance
- **Value**: Insert additional tests only for high-risk devices, balance coverage vs cost with engineering confidence

## 🔄 Model Interpretability Workflow

```mermaid
graph TB
    A[Trained Model] --> B{Interpretation Goal?}
    
    B -->|Global Understanding| C[Feature Importance]
    B -->|Local Explanation| D[Single Prediction]
    
    C --> E[Permutation Importance]
    C --> F[SHAP Summary Plot]
    C --> G[Partial Dependence]
    
    D --> H[SHAP Force Plot]
    D --> I[LIME Explanation]
    D --> J[ICE Curves]
    
    E --> K[Validate with Domain Expert]
    F --> K
    G --> K
    H --> L[Debug Prediction]
    I --> L
    J --> L
    
    K --> M{Makes Sense?}
    L --> N{Trustworthy?}
    
    M -->|No| O[Refine Model]
    M -->|Yes| P[Deploy with Confidence]
    N -->|No| O
    N -->|Yes| P
    
    style A fill:#e1f5ff
    style P fill:#e1ffe1
    style O fill:#ffe1e1
```

## 📊 Learning Path Context

**Prerequisites:**
- **010**: Linear Regression - Coefficient interpretation
- **016**: Decision Trees - Tree-based feature importance
- **022**: Random Forest - Ensemble feature importance
- **103**: Advanced Feature Engineering - Understanding engineered features

**This Notebook (104):**
- Permutation importance (model-agnostic)
- SHAP values (Shapley values from game theory)
- LIME (local linear approximations)
- Partial Dependence Plots (PDP) and Individual Conditional Expectation (ICE)
- Feature importance validation and comparison

**Next Steps:**
- **105**: AutoML & NAS - Automated model selection with interpretability constraints
- **107**: Model Monitoring - Detecting when feature importance shifts in production
- **054**: Advanced Deep Learning - Attention mechanisms and neural network interpretability

---

Let's make black-box models transparent and trustworthy! 🔍

## 1. Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
import warnings
warnings.filterwarnings('ignore')

# Interpretability libraries
# Note: Install with: pip install shap lime
try:
    import shap
    print("✅ SHAP available")
except ImportError:
    print("⚠️  Install SHAP: pip install shap")

try:
    import lime
    from lime import lime_tabular
    print("✅ LIME available")
except ImportError:
    print("⚠️  Install LIME: pip install lime")

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Random seed
np.random.seed(42)

print("\n✅ Environment ready for model interpretability analysis!")

## 2. Generate Semiconductor Test Data

**Purpose:** Create synthetic STDF parametric test data with known feature-target relationships.

**Key Points:**
- **Non-linear relationships**: Yield depends on power (vdd×idd) and thermal stress (temp×freq)
- **Interaction effects**: High Vdd + low Vth = increased leakage, lower yield
- **Spatial patterns**: Die position affects yield (wafer edge effect)
- **Why this matters**: We know ground truth, can validate if interpretability methods recover it

In [ ]:
# Generate 2000 devices from wafer test
n_devices = 2000

# Parametric measurements (STDF-like)
vdd = np.random.normal(1.2, 0.08, n_devices)   # Supply voltage (V)
idd = np.random.normal(50, 8, n_devices)       # Supply current (mA)
freq = np.random.normal(2000, 150, n_devices)  # Max frequency (MHz)
temp = np.random.normal(85, 12, n_devices)     # Junction temp (°C)
vth = np.random.normal(0.4, 0.03, n_devices)   # Threshold voltage (V)

# Spatial coordinates
die_x = np.random.randint(0, 25, n_devices)
die_y = np.random.randint(0, 25, n_devices)
radius = np.sqrt((die_x - 12)**2 + (die_y - 12)**2)  # Distance from center

# Ground truth relationships (what we want interpretability to discover)
power = vdd * idd  # Physical law: P = V × I
thermal_stress = temp * freq / 1000  # Thermal stress index
overdrive = vdd - vth  # Overdrive voltage affects speed

# Target: Yield score (influenced by multiple factors)
yield_score = (
    100  # Base yield
    - 0.4 * power  # High power reduces yield
    - 0.015 * thermal_stress  # Thermal stress degrades reliability
    + 8 * vth  # Higher Vth = better reliability
    - 0.3 * radius  # Edge dies have lower yield
    - 5 * (vdd > 1.3) * (vth < 0.38)  # Interaction: high V + low Vth = leakage
    + np.random.normal(0, 2.5, n_devices)  # Measurement noise
)
yield_score = np.clip(yield_score, 65, 100)

# Create DataFrame
df = pd.DataFrame({
    'vdd': vdd,
    'idd': idd,
    'freq': freq,
    'temp': temp,
    'vth': vth,
    'die_x': die_x,
    'die_y': die_y,
    'radius': radius,
    'yield': yield_score
})

print(f"Dataset: {df.shape[0]} devices, {df.shape[1]} columns")
print(f"\nYield statistics:")
print(df['yield'].describe())
print(f"\nFeature correlations with yield:")
print(df.corr()['yield'].sort_values(ascending=False))

## 3. Train Random Forest Model

**Purpose:** Build a non-linear model that we'll interpret using multiple techniques.

**Key Points:**
- **Random Forest**: Complex ensemble model (not easily interpretable like linear regression)
- **High accuracy**: Should achieve R² > 0.85 given strong signal in data
- **Black box problem**: 100 trees × 20 max depth = impossible to manually interpret
- **Why this matters**: Real production models are complex; need interpretability tools to understand them

In [ ]:
# Select features (exclude radius as it's derived)
feature_cols = ['vdd', 'idd', 'freq', 'temp', 'vth', 'die_x', 'die_y']
X = df[feature_cols]
y = df['yield']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("Random Forest Model Performance:")
print(f"  Train R²: {train_r2:.4f}")
print(f"  Test R²: {test_r2:.4f}")
print(f"  Test RMSE: {test_rmse:.2f}%")
print(f"\n  Model Complexity: {rf_model.n_estimators} trees × max depth {rf_model.max_depth}")
print(f"  Total leaf nodes: ~{rf_model.n_estimators * (2**rf_model.max_depth)} (impossible to interpret manually!)")

## 4. Method 1: Tree-Based Feature Importance (Built-in)

**Concept:** Average impurity decrease when splitting on each feature across all trees.

**Mathematics:**
$$\text{Importance}(f) = \frac{1}{N_{trees}} \sum_{t=1}^{N_{trees}} \sum_{nodes\ using\ f} \Delta \text{Impurity}$$

**Pros:** Fast, built into sklearn  
**Cons:** Biased toward high-cardinality features, unreliable with correlated features

In [ ]:
# Extract built-in feature importance
tree_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Tree-Based Feature Importance (Gini Impurity):")
print(tree_importance.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 5))
plt.barh(range(len(tree_importance)), tree_importance['importance'])
plt.yticks(range(len(tree_importance)), tree_importance['feature'])
plt.xlabel('Gini Importance')
plt.title('Random Forest Built-in Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print(f"  Top feature: {tree_importance.iloc[0]['feature']} (importance = {tree_importance.iloc[0]['importance']:.3f})")
print(f"  Model relies most on {tree_importance.iloc[0]['feature']} for split decisions")

## 5. Method 2: Permutation Importance (Model-Agnostic)

**Concept:** Shuffle each feature and measure performance drop. Larger drop = more important.

**Mathematics:**
$$\text{Importance}(f) = \text{Score}_{original} - \text{Score}_{f\ permuted}$$

**Pros:** Unbiased, works with any model  
**Cons:** Slower (requires retraining), can be unstable with correlated features

In [ ]:
# Compute permutation importance on test set
perm_importance = permutation_importance(
    rf_model, X_test, y_test,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

# Create DataFrame
perm_importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("Permutation Importance (R² drop when shuffled):")
print(perm_importance_df.to_string(index=False))

# Visualize with error bars
plt.figure(figsize=(10, 5))
plt.barh(
    range(len(perm_importance_df)),
    perm_importance_df['importance_mean'],
    xerr=perm_importance_df['importance_std'],
    capsize=5
)
plt.yticks(range(len(perm_importance_df)), perm_importance_df['feature'])
plt.xlabel('Importance (R² Drop)')
plt.title('Permutation Importance (10 repeats)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print(f"  If we randomize {perm_importance_df.iloc[0]['feature']}, R² drops by {perm_importance_df.iloc[0]['importance_mean']:.4f}")
print(f"  This is the most impactful feature for model performance")

## 6. Method 3: SHAP Values (Shapley Additive Explanations)

**Concept:** Game theory approach—compute each feature's marginal contribution to prediction.

**Mathematics (Shapley Value):**
$$\phi_i = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(|N|-|S|-1)!}{|N|!} [f(S \cup \{i\}) - f(S)]$$

**Properties:**
- **Additivity**: Sum of SHAP values = prediction - baseline
- **Consistency**: If feature helps more, SHAP value increases
- **Local accuracy**: Exact attribution for each prediction

In [ ]:
# Create SHAP explainer (TreeExplainer for tree models)
explainer = shap.TreeExplainer(rf_model)

# Calculate SHAP values for test set (use sample for speed)
X_test_sample = X_test.sample(n=min(500, len(X_test)), random_state=42)
shap_values = explainer.shap_values(X_test_sample)

print(f"SHAP values computed for {X_test_sample.shape[0]} test samples")
print(f"SHAP values shape: {shap_values.shape}")
print(f"\nBase value (expected model output): {explainer.expected_value:.2f}%")
print(f"Actual predictions range: {rf_model.predict(X_test_sample).min():.2f}% to {rf_model.predict(X_test_sample).max():.2f}%")

## 7. SHAP Summary Plot (Global Importance)

**Purpose:** Visualize feature importance AND feature effects.

**Key Points:**
- **X-axis**: SHAP value (impact on prediction)
- **Y-axis**: Features ranked by importance
- **Color**: Feature value (red = high, blue = low)
- **Why powerful**: Shows not just importance, but also direction of effect

In [ ]:
# SHAP summary plot
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, X_test_sample, show=False)
plt.tight_layout()
plt.show()

# Calculate mean absolute SHAP values (global importance)
shap_importance = pd.DataFrame({
    'feature': feature_cols,
    'mean_abs_shap': np.abs(shap_values).mean(axis=0)
}).sort_values('mean_abs_shap', ascending=False)

print("\nSHAP Global Importance (mean |SHAP value|):")
print(shap_importance.to_string(index=False))

print("\n📊 Interpretation:")
print(f"  {shap_importance.iloc[0]['feature']} has highest average impact: {shap_importance.iloc[0]['mean_abs_shap']:.3f}")
print(f"  Red dots (high {shap_importance.iloc[0]['feature']}) push predictions up/down")
print(f"  Blue dots (low {shap_importance.iloc[0]['feature']}) have opposite effect")

## 8. SHAP Force Plot (Single Prediction Explanation)

**Purpose:** Explain one specific prediction—why did Device #X get this yield score?

**Key Points:**
- **Base value**: Average model prediction (baseline)
- **Red arrows**: Features pushing prediction higher
- **Blue arrows**: Features pushing prediction lower
- **Final value**: Actual prediction = base + sum of SHAP values

In [ ]:
# Select an interesting device to explain (low yield)
sample_idx = X_test_sample['vdd'].idxmax()  # Device with highest Vdd
sample_device = X_test_sample.loc[sample_idx]
sample_prediction = rf_model.predict(sample_device.values.reshape(1, -1))[0]
sample_shap = shap_values[X_test_sample.index.get_loc(sample_idx)]

print(f"Explaining Device #{sample_idx}:")
print(f"\nFeature values:")
for feat, val in sample_device.items():
    print(f"  {feat}: {val:.3f}")
print(f"\nPredicted yield: {sample_prediction:.2f}%")
print(f"Baseline (average): {explainer.expected_value:.2f}%")
print(f"Difference: {sample_prediction - explainer.expected_value:.2f}%")

# SHAP force plot
shap.force_plot(
    explainer.expected_value,
    sample_shap,
    sample_device,
    matplotlib=True,
    show=False
)
plt.tight_layout()
plt.show()

# Show top contributing features
feature_contributions = pd.DataFrame({
    'feature': feature_cols,
    'value': sample_device.values,
    'shap_value': sample_shap
}).sort_values('shap_value', key=abs, ascending=False)

print("\nTop 3 Contributing Features:")
for i in range(3):
    row = feature_contributions.iloc[i]
    direction = "increasing" if row['shap_value'] > 0 else "decreasing"
    print(f"  {i+1}. {row['feature']} = {row['value']:.3f} → {direction} yield by {abs(row['shap_value']):.2f}%")

## 9. Method 4: LIME (Local Interpretable Model-agnostic Explanations)

**Concept:** Fit a simple linear model locally around the prediction to approximate behavior.

**Mathematics:**
$$\text{explanation}(x) = \arg\min_{g \in G} L(f, g, \pi_x) + \Omega(g)$$

Where:
- $f$ = complex model
- $g$ = simple linear model
- $\pi_x$ = proximity to instance $x$
- $\Omega(g)$ = complexity penalty

**Why useful:** Approximates any model with interpretable linear model locally

In [ ]:
# Create LIME explainer
lime_explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=feature_cols,
    mode='regression',
    random_state=42
)

# Explain the same device we used for SHAP
lime_explanation = lime_explainer.explain_instance(
    data_row=sample_device.values,
    predict_fn=rf_model.predict,
    num_features=len(feature_cols)
)

print(f"LIME Explanation for Device #{sample_idx}:")
print(f"\nPrediction: {sample_prediction:.2f}%")
print(f"Local linear model R²: {lime_explanation.score:.3f}")
print(f"\nLinear coefficients (local approximation):")

# Extract feature contributions
lime_features = lime_explanation.as_list()
for feature_desc, weight in lime_features:
    direction = "↑" if weight > 0 else "↓"
    print(f"  {feature_desc}: {weight:+.3f} {direction}")

# Visualize LIME explanation
fig = lime_explanation.as_pyplot_figure()
plt.title(f"LIME Explanation: Device #{sample_idx}")
plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print(f"  LIME fitted a linear model locally (R²={lime_explanation.score:.3f})")
print(f"  This linear model approximates Random Forest's behavior near this device")
print(f"  Compare LIME weights to SHAP values for validation")

## 10. Partial Dependence Plots (PDPs)

**Concept:** Show the marginal effect of a feature on predictions, averaging over all other features.

**Mathematics:**
$$\text{PDP}(x_s) = \mathbb{E}_{x_c}[f(x_s, x_c)] = \frac{1}{n} \sum_{i=1}^{n} f(x_s, x_c^{(i)})$$

Where $x_s$ = feature of interest, $x_c$ = all other features

**Why useful:** Reveals non-linear relationships and interaction effects

In [ ]:
# Create partial dependence plots for top features
top_features = shap_importance.head(4)['feature'].tolist()
feature_indices = [feature_cols.index(f) for f in top_features]

fig, ax = plt.subplots(2, 2, figsize=(14, 10))
display = PartialDependenceDisplay.from_estimator(
    rf_model,
    X_train,
    features=feature_indices,
    feature_names=feature_cols,
    ax=ax.ravel(),
    n_jobs=-1
)
plt.suptitle('Partial Dependence Plots (Top 4 Features)', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("  PDP shows average model response when varying one feature")
print("  Flat line = feature has no effect")
print("  Curved line = non-linear relationship")
print("  Example: If vdd PDP slopes downward, higher voltage → lower yield (as expected from power equation)")

## 11. Compare All Importance Methods

**Purpose:** Validate consistency across different interpretability techniques.

**Key Points:**
- **Agreement**: If all methods rank same features top, high confidence
- **Disagreement**: Investigate—could indicate correlated features or method-specific biases
- **Production strategy**: Use ensemble of methods, not single technique

In [ ]:
# Normalize importance scores to [0, 1] for comparison
def normalize_importance(values):
    return (values - values.min()) / (values.max() - values.min())

# Combine all importance methods
comparison_df = pd.DataFrame({
    'feature': feature_cols,
    'tree_importance': normalize_importance(tree_importance.set_index('feature').loc[feature_cols]['importance'].values),
    'permutation': normalize_importance(perm_importance_df.set_index('feature').loc[feature_cols]['importance_mean'].values),
    'shap_mean_abs': normalize_importance(shap_importance.set_index('feature').loc[feature_cols]['mean_abs_shap'].values)
})

# Calculate average rank
comparison_df['avg_importance'] = comparison_df[['tree_importance', 'permutation', 'shap_mean_abs']].mean(axis=1)
comparison_df = comparison_df.sort_values('avg_importance', ascending=False)

print("Feature Importance Comparison (normalized to 0-1):")
print(comparison_df.to_string(index=False, float_format='%.3f'))

# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(feature_cols))
width = 0.25

ax.bar(x - width, comparison_df['tree_importance'], width, label='Tree (Gini)', alpha=0.8)
ax.bar(x, comparison_df['permutation'], width, label='Permutation', alpha=0.8)
ax.bar(x + width, comparison_df['shap_mean_abs'], width, label='SHAP', alpha=0.8)

ax.set_xlabel('Features')
ax.set_ylabel('Normalized Importance')
ax.set_title('Feature Importance Method Comparison')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['feature'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate rank correlation
print("\n📊 Method Agreement:")
from scipy.stats import spearmanr
corr_tree_perm = spearmanr(comparison_df['tree_importance'], comparison_df['permutation'])[0]
corr_tree_shap = spearmanr(comparison_df['tree_importance'], comparison_df['shap_mean_abs'])[0]
corr_perm_shap = spearmanr(comparison_df['permutation'], comparison_df['shap_mean_abs'])[0]

print(f"  Tree vs Permutation: ρ = {corr_tree_perm:.3f}")
print(f"  Tree vs SHAP: ρ = {corr_tree_shap:.3f}")
print(f"  Permutation vs SHAP: ρ = {corr_perm_shap:.3f}")
print(f"\n  Average correlation: {np.mean([corr_tree_perm, corr_tree_shap, corr_perm_shap]):.3f}")
print(f"  ✅ High agreement (ρ > 0.8) = trustworthy feature importance rankings")

## 12. Comprehensive Visualization Dashboard

In [ ]:
# Create comprehensive interpretability dashboard
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Predictions vs Actual
ax1 = fig.add_subplot(gs[0, 0])
ax1.scatter(y_test, y_pred_test, alpha=0.5, s=20)
ax1.plot([65, 100], [65, 100], 'r--', lw=2)
ax1.set_xlabel('Actual Yield (%)')
ax1.set_ylabel('Predicted Yield (%)')
ax1.set_title(f'Model Performance (R²={test_r2:.3f})')
ax1.grid(True, alpha=0.3)

# 2. Feature Importance Comparison
ax2 = fig.add_subplot(gs[0, 1:])
x_pos = np.arange(len(comparison_df))
ax2.barh(x_pos - 0.25, comparison_df['tree_importance'], 0.25, label='Tree', alpha=0.8)
ax2.barh(x_pos, comparison_df['permutation'], 0.25, label='Permutation', alpha=0.8)
ax2.barh(x_pos + 0.25, comparison_df['shap_mean_abs'], 0.25, label='SHAP', alpha=0.8)
ax2.set_yticks(x_pos)
ax2.set_yticklabels(comparison_df['feature'])
ax2.set_xlabel('Normalized Importance')
ax2.set_title('Feature Importance Methods')
ax2.legend(loc='lower right')
ax2.invert_yaxis()

# 3. SHAP Summary (embedded)
ax3 = fig.add_subplot(gs[1, :])
shap.summary_plot(shap_values, X_test_sample, show=False, plot_size=None)
ax3.set_title('SHAP Feature Effects')

# 4. Single prediction breakdown
ax4 = fig.add_subplot(gs[2, :2])
feature_contributions_sorted = feature_contributions.sort_values('shap_value')
colors = ['red' if x > 0 else 'blue' for x in feature_contributions_sorted['shap_value']]
ax4.barh(range(len(feature_contributions_sorted)), feature_contributions_sorted['shap_value'], color=colors, alpha=0.7)
ax4.set_yticks(range(len(feature_contributions_sorted)))
ax4.set_yticklabels(feature_contributions_sorted['feature'])
ax4.set_xlabel('SHAP Value (% yield impact)')
ax4.set_title(f'Device #{sample_idx} Explanation (predicted: {sample_prediction:.1f}%)')
ax4.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax4.grid(axis='x', alpha=0.3)

# 5. Feature value distribution
ax5 = fig.add_subplot(gs[2, 2])
top_feature = comparison_df.iloc[0]['feature']
ax5.hist(X_train[top_feature], bins=30, alpha=0.7, edgecolor='black')
ax5.axvline(sample_device[top_feature], color='red', linestyle='--', linewidth=2, label=f'Device #{sample_idx}')
ax5.set_xlabel(top_feature)
ax5.set_ylabel('Frequency')
ax5.set_title(f'Distribution: {top_feature}')
ax5.legend()
ax5.grid(axis='y', alpha=0.3)

plt.suptitle('Model Interpretability Dashboard', fontsize=16, y=0.995)
plt.show()

## 13. Project Templates

### Project 1: Automated Yield Loss Root Cause System
**Objective:** Build real-time SHAP-based root cause identification for failed devices
- Load real STDF wafer test data with 200+ parametric tests
- Train binary classifier (pass/fail) with Random Forest or XGBoost
- For each failed device, compute SHAP values and rank top 5 contributing parameters
- Generate automated report: "Device failed due to: 1) Idd (+3.2σ), 2) Freq (-1.8σ), 3) Temp (+2.1σ)"
- **Success Metric:** Reduce root cause analysis time from 2 weeks to <2 hours, validate with fab engineers on 10 real yield excursions

### Project 2: Interpretable Test Time Prediction
**Objective:** Explain why specific devices take longer to test
- Use test time (seconds) as target variable
- Features: device type, lot characteristics, ATE platform, time-of-day, test sequence
- Apply LIME to explain outliers (devices taking >2× median time)
- Identify systematic issues (e.g., "Lot B always slow on ATE-7 due to calibration drift")
- **Success Metric:** Find 3+ actionable insights leading to 10% average test time reduction

### Project 3: Trusted Bin Prediction with Explanation
**Objective:** Deploy bin prediction model with per-device explanations for engineers
- Multi-class classification: BIN1 (premium), BIN2 (standard), BIN7 (fail), BIN9 (retest)
- Train ensemble model (Random Forest or LightGBM)
- Generate SHAP force plots for every prediction in production
- Create dashboard showing: prediction, confidence, top 3 reasons, similar historical devices
- **Success Metric:** >80% engineer trust score, <5% override rate, feedback validates SHAP reasoning

### Project 4: Feature Engineering Validation via Interpretability
**Objective:** Verify engineered features are being used correctly by the model
- Create 20+ engineered features (interactions, polynomials, domain-specific)
- Train model and compute SHAP importance for engineered vs raw features
- Check if power (vdd×idd) has higher importance than vdd, idd individually
- Prune low-SHAP-value features (<0.01 mean |SHAP|)
- **Success Metric:** Reduce feature count 40% while maintaining R² within 1%, improved model latency

### Project 5: Bias Detection in Wafer-Level Predictions
**Objective:** Use interpretability to detect and remove spatial bias in yield models
- Train model predicting yield from parametric tests + spatial coordinates
- Compute SHAP values for die_x, die_y, radius features
- If spatial features dominate (>30% SHAP importance), model learned wafer map patterns
- Retrain with spatial features removed, validate performance doesn't drop
- **Success Metric:** Model generalizes to new lots without overfitting to historical spatial patterns

### Project 6: Partial Dependence for Spec Limit Optimization
**Objective:** Use PDPs to set optimal parametric test limits
- Train yield prediction model from parametric tests
- Generate PDP for each test parameter showing yield vs parameter value
- Identify inflection points where yield drops sharply (new spec limit candidates)
- Compare PDP-derived limits to current data sheet specs
- **Success Metric:** Propose 5+ spec limit changes backed by data, simulate 2-5% yield improvement

### Project 7: LIME-Based Anomaly Explanation
**Objective:** Explain anomalous devices using local linear approximations
- Train autoencoder or isolation forest for anomaly detection on parametric data
- For flagged anomalies, use LIME to explain reconstruction error or anomaly score
- Generate human-readable explanations: "Anomalous because: vth=-2.3σ, idd=+3.1σ, freq=-1.2σ"
- Cross-reference with SHAP explanations for validation
- **Success Metric:** 90% of LIME-flagged anomaly reasons confirmed by fab engineers as real issues

### Project 8: Interactive Interpretability Dashboard
**Objective:** Build Streamlit/Dash app for real-time model interpretation
- Upload STDF file → predict yield/bin for all devices
- Click any device → show SHAP force plot, LIME explanation, feature values
- Compare multiple devices side-by-side with SHAP value differences
- Export explanations to PDF reports for management
- **Success Metric:** Deployed to 5+ test engineers, used weekly for production debug, <5min time-to-insight

## 🎓 Key Takeaways

**When to Use Each Method:**
- **Tree-based importance**: Quick baseline for tree models (RF, XGBoost), but biased
- **Permutation importance**: Model-agnostic, unbiased, works for any model
- **SHAP values**: Gold standard—both global and local explanations, theoretically sound
- **LIME**: When you need simple linear approximations, fast for single predictions
- **PDPs**: Understanding non-linear feature effects, but averages over interactions

**Limitations:**
- ⚠️ **Correlated features**: All methods struggle when features are highly correlated (multicollinearity)
- ⚠️ **Computational cost**: SHAP can be slow for large datasets (use TreeExplainer or sampling)
- ⚠️ **Interpretation != Causation**: High importance doesn't mean causal relationship
- ⚠️ **Adversarial explanations**: Models can be trained to give misleading explanations
- ⚠️ **Human bias**: Engineers may cherry-pick explanations that confirm existing beliefs

**Best Practices:**
1. **Use multiple methods**: If SHAP, permutation, and tree importance agree → high confidence
2. **Validate with domain experts**: Do top features match physics/engineering knowledge?
3. **Check for data leakage**: If importance seems too good, investigate feature-target leakage
4. **Sample for speed**: SHAP on 500-1000 samples usually sufficient for global patterns
5. **Document explanations**: Include SHAP plots in model cards for production transparency
6. **Monitor in production**: Feature importance can shift with data drift—re-compute monthly
7. **Use explanations for debugging**: If model fails, SHAP shows which features caused error

**Production Checklist:**
- [ ] Compute SHAP values on validation set
- [ ] Verify top 5 features make engineering sense
- [ ] Create SHAP summary plot for model card
- [ ] Build API endpoint for single-prediction explanations
- [ ] Set up monitoring for feature importance drift
- [ ] Train stakeholders on reading SHAP plots
- [ ] Document known limitations and failure modes

**Next Steps:**
- Study **105: AutoML & NAS** for automated model search with interpretability constraints
- Explore **107: Model Monitoring** to detect when feature importance changes in production
- Apply to **Deep Learning (054)** with attention mechanisms and gradient-based explanations
- Read "Interpretable Machine Learning" by Christoph Molnar (free online book)
- Experiment with SHAP's advanced plots: dependence plots, interaction plots, waterfall plots